# Active New Building Construction Sites

## Introduction

In this notebook we attempt to get a handle on data for active new building construction sites in New York City.

In [1]:
import requests
import pandas as pd
pd.set_option("max_columns", 500)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import geopandas as gpd
from datetime import datetime
import co_reader

## Download

In [2]:
def download_file(url, filename):
    """
    Helper method handling downloading large files from `url` to `filename`. Returns a pointer to `filename`.
    """
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return filename

In [3]:
permits = download_file("https://data.cityofnewyork.us/api/views/ipu4-2q9a/rows.csv?accessType=DOWNLOAD",
                        "DOB Permit Issuance.csv")
permits = pd.read_csv(permits)

/Users/Honors/anaconda/envs/nyc-construction-timelines/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,8,9,11,12,31,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Preprocessing

We need to find construction permits corresponding with new building jobs which have yet to expire.

We start by filtering those down and converting the issuance and expiration dates from strings to intelligent datetimes.

In [4]:
nb_permits = permits[(permits['Job Type'] == 'NB') &
                     (permits['Permit Type'] == 'NB') &
                     (permits['Permit Status'] == 'ISSUED')]

In [5]:
nb_permits['Issuance Date'] = nb_permits['Issuance Date'].map(lambda date: pd.to_datetime(date))
nb_permits['Expiration Date'] = nb_permits['Expiration Date'].map(lambda date: pd.to_datetime(date))

/Users/Honors/anaconda/envs/nyc-construction-timelines/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Honors/anaconda/envs/nyc-construction-timelines/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Because of technical differences in PDF link name formatting, `co_reader` requires that you pass it the DOB borough code for the building. To simplify this operation let's remap a borough code column for the entire dataset of interest, using the existing borough column.

In [6]:
borough_mapper = {
    "MANHATTAN": "M",
    "BROOKLYN": "B",
    "QUEENS": "Q",
    "STATEN ISLAND": "R",
    "BRONX": "X"
}

nb_permits['Borough Code'] = nb_permits['BOROUGH'].map(lambda b: borough_mapper[b])

/Users/Honors/anaconda/envs/nyc-construction-timelines/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


How many new building construction permits are active right now?

In [8]:
now = pd.to_datetime(datetime.now())

In [9]:
(nb_permits['Expiration Date'] > now).astype(int).sum()

4961

In [10]:
bins_with_nonexpired_permits = nb_permits[nb_permits['Expiration Date'] > now]['Bin #'].astype(int).unique()

How many unique lots have currently-active new building construction permits?

This filters out lots which have recieved multiple permits (reissuance etc.) which are still valid.

In [11]:
len(bins_with_nonexpired_permits)

4083

We will need to start time for each of these permits, as this is what we will be comparing against in order to determine whether or not a building has finished construction. We will also need the borough code that we just generated.

The loop that follows the selects and takes this information off of the most recent new building document on record&mdash;the one with the highest permit number.

Even we do not filter the data this way we will recirculate each of the thousand outstanding "additional" permits, increasing runtime by 20%. Even though the end result would be the same, given how long certificate data reads take, this is wasteful, so it is worth the additional work of removing these explicitly beforehand.

In [12]:
now = pd.to_datetime(datetime.now())

In [13]:
active_nb_permits = nb_permits[nb_permits['Expiration Date'] > now]

In [14]:
active_nb_permits.head(0)

,BOROUGH,Bin #,House #,Street Name,Job #,Job doc. #,Job Type,Self_Cert,Block,Lot,Community Board,Zip Code,Bldg Type,Residential,Special District 1,Special District 2,Work Type,Permit Status,Filing Status,Permit Type,Permit Sequence #,Permit Subtype,Oil Gas,Site Fill,Filing Date,Issuance Date,Expiration Date,Job Start Date,Permittee's First Name,Permittee's Last Name,Permittee's Business Name,Permittee's Phone #,Permittee's License Type,Permittee's License #,Act as Superintendent,Permittee's Other Title,HIC License,Site Safety Mgr's First Name,Site Safety Mgr's Last Name,Site Safety Mgr Business Name,Superintendent First & Last Name,Superintendent Business Name,Owner's Business Type,Non-Profit,Owner's Business Name,Owner's First Name,Owner's Last Name,Owner's House #,Owner's House Street Name,Owner’s House City,Owner’s House State,Owner’s House Zip Code,Owner's Phone #,DOBRunDate,Borough Code


In [15]:
most_recent_docs = []
nb_permits['Bin #'] = nb_permits['Bin #'].astype(int)
bins_with_nonexpired_permits = nb_permits[nb_permits['Expiration Date'] > now]['Bin #'].unique()
active_nb_permits = nb_permits[nb_permits['Expiration Date'] > now]

for BIN in bins_with_nonexpired_permits:
    docs = active_nb_permits[active_nb_permits['Bin #'] == BIN]
    doc = docs.iloc[np.argmax(docs['Permit Sequence #'].values)]
    most_recent_docs.append(doc)

active_latest_nb_permits = pd.concat(most_recent_docs, axis=1).T

/Users/Honors/anaconda/envs/nyc-construction-timelines/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [16]:
len(active_latest_nb_permits)

4083

That this number is exactly the same as the number of unique BINs before confirms that the routine fired successfully.

In [17]:
pd.to_pickle(active_latest_nb_permits, "Latest Active New Building Permits.p")

## Processing

4083 Certificate of Occupancy reads will take a long time to process. This step must be handled in segments.

In [2]:
active_latest_nb_permits = pd.read_pickle("Latest Active New Building Permits.p")

In [5]:
active_latest_nb_permits.head(1)

,BOROUGH,Bin #,House #,Street Name,Job #,Job doc. #,Job Type,Self_Cert,Block,Lot,Community Board,Zip Code,Bldg Type,Residential,Special District 1,Special District 2,Work Type,Permit Status,Filing Status,Permit Type,Permit Sequence #,Permit Subtype,Oil Gas,Site Fill,Filing Date,Issuance Date,Expiration Date,Job Start Date,Permittee's First Name,Permittee's Last Name,Permittee's Business Name,Permittee's Phone #,Permittee's License Type,Permittee's License #,Act as Superintendent,Permittee's Other Title,HIC License,Site Safety Mgr's First Name,Site Safety Mgr's Last Name,Site Safety Mgr Business Name,Superintendent First & Last Name,Superintendent Business Name,Owner's Business Type,Non-Profit,Owner's Business Name,Owner's First Name,Owner's Last Name,Owner's House #,Owner's House Street Name,Owner’s House City,Owner’s House State,Owner’s House Zip Code,Owner's Phone #,DOBRunDate,Borough Code
566401,BROOKLYN,3034837,329,STERLING ST.,3.20708e+08,1,NB,NaN,1316,72,309,11225,2,YES,NaN,NaN,,ISSUED,RENEWAL,NB,4,,,OFF-SITE,07/08/2016,2016-07-08 00:00:00,2016-11-25 00:00:00,07/07/2014,ZEV,CHASKELSON,HML DEVELOPMENTS LLC,7187021530,GENERAL CONTRACTOR,0613324,,,,,,,ZEV CHASKELSON,ZENCO GROUP INC,PARTNERSHIP,NaN,JACQUELYN 327 LLC,AL,LIEBER,146,SPENCER STREET,BROOKLYN,NY,11205,3472274450,07/09/2016 12:00:00 AM,B


In [3]:
def latest_co_date(srs):
    """
    DataFrame apply function which retrieves and stores the most recent found C/O date in the DataFrame.
    """
    try:
        return co_reader.get_co_date(srs['Bin #'], srs['Borough Code'])
    except Exception as e:
        print("WARNING: Error raised:\n", e)
        return None

    
def is_active(srs):
    """
    DataFrame apply function which retrieves and stores whether or not a construction site is active.
    
    Uses the "Latest C/O Date" field specified by the `latest_co_date` function above.
    """
    if srs['Latest C/O Date']:
        if srs['Latest C/O Date'].replace(tzinfo=None) > srs['Issuance Date'].replace(tzinfo=None):
            return False
        else:
            return True
    else:
        return True

Now we run the primary algorithm---in 100-permit segments, for runtime splitting sake.

Because of the verbocity of the logging output, while this script was running I temporarily commented out the print statements in the `co_reader` script.

In [ ]:
active_sample_1 = active_latest_nb_permits.iloc[0:100]
active_sample_1['Latest C/O Date'] = active_sample_1.apply(latest_co_date, axis='columns')
active_sample_1['Active Construction Site'] = active_sample_1.apply(is_active, axis='columns')

Requested BIN 3034837 data from BIS, awaiting response...


^ Latest.

In [5]:
active_sample_2 = active_latest_nb_permits.iloc[100:200]
active_sample_2['Latest C/O Date'] = active_sample_2.apply(latest_co_date, axis='columns')
active_sample_2['Active Construction Site'] = active_sample_2.apply(is_active, axis='columns')

/Users/Honors/anaconda/envs/nyc-construction-timelines/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Honors/anaconda/envs/nyc-construction-timelines/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [6]:
active_sample_3 = active_latest_nb_permits.iloc[200:300]
active_sample_3['Latest C/O Date'] = active_sample_3.apply(latest_co_date, axis='columns')
active_sample_3['Active Construction Site'] = active_sample_3.apply(is_active, axis='columns')

/Users/Honors/anaconda/envs/nyc-construction-timelines/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Honors/anaconda/envs/nyc-construction-timelines/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [14]:
active_sample_4 = active_latest_nb_permits.iloc[300:400]
active_sample_4['Latest C/O Date'] = active_sample_4.apply(latest_co_date, axis='columns')
active_sample_4['Active Construction Site'] = active_sample_4

In [ ]:
active_sample_5 = active_latest_nb_permits.iloc[400:500]
active_sample_5['Latest C/O Date'] = active_sample_5.apply(latest_co_date, axis='columns')
active_sample_5['Active Construction Site'] = active_sample_5

In [ ]:
active_sample_6 = active_latest_nb_permits.iloc[500:600]
active_sample_6['Latest C/O Date'] = active_sample_6.apply(latest_co_date, axis='columns')
active_sample_6['Active Construction Site'] = active_sample_6

In [ ]:
active_sample_7 = active_latest_nb_permits.iloc[600:700]
active_sample_7['Latest C/O Date'] = active_sample_7.apply(latest_co_date, axis='columns')
active_sample_7['Active Construction Site'] = active_sample_7

In [ ]:
active_sample_8 = active_latest_nb_permits.iloc[700:800]
active_sample_8['Latest C/O Date'] = active_sample_6.apply(latest_co_date, axis='columns')
active_sample_8['Active Construction Site'] = active_sample_8

In [ ]:
active_sample_9 = active_latest_nb_permits.iloc[800:900]
active_sample_9['Latest C/O Date'] = active_sample_9.apply(latest_co_date, axis='columns')
active_sample_9['Active Construction Site'] = active_sample_9

In [ ]:
active_sample_10 = active_latest_nb_permits.iloc[900:1000]
active_sample_10['Latest C/O Date'] = active_sample_10.apply(latest_co_date, axis='columns')
active_sample_10['Active Construction Site'] = active_sample_10

In [ ]:
active_sample_11 = active_latest_nb_permits.iloc[1000:1100]
active_sample_11['Latest C/O Date'] = active_sample_11.apply(latest_co_date, axis='columns')
active_sample_11['Active Construction Site'] = active_sample_11

In [ ]:
active_sample_12 = active_latest_nb_permits.iloc[1100:1200]
active_sample_12['Latest C/O Date'] = active_sample_12.apply(latest_co_date, axis='columns')
active_sample_12['Active Construction Site'] = active_sample_12

In [ ]:
active_sample_13 = active_latest_nb_permits.iloc[1200:1300]
active_sample_13['Latest C/O Date'] = active_sample_13.apply(latest_co_date, axis='columns')
active_sample_13['Active Construction Site'] = active_sample_13

In [40]:
# active_sample_2['Latest C/O Date'] = active_sample_2.apply(latest_co_date, axis='columns')